In [59]:
import pandas as pd
import psycopg2
import matplotlib
import numpy as np
#from connect import connect_DB #import custom connection script

#connector = connect_DB() #instantiate connector class
conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
cur = conn.cursor()
#list of HMDA years to use
#property type appears in 2004
#FIXME reload and clean 2004
#FIXME: add property type into 2004 and beyond?
hmda_tables = ['hmdalar2000', 'hmdalar2001', 'hmdalar2002', 'hmdalar2003', 'hmdalar2004', 'hmdalar2005', 'hmdalar2006',
'hmdalar2007', 'hmdalar2008', 'hmdalar2009', 'hmdalar2010', 'hmdalar2011', 'hmdalar2012', 'hmdalar2013', 'hmdalar2014']


#get county level data
#count of loans
#average and median dollar value of loans
#dollar volume of loan amount


In [60]:
#selects application information for single family homes
#FIXME add section for origination only
first = True
for table in hmda_tables:

    SQL = """
  WITH 
  county_apps AS (
  SELECT year, state, county,
    ROUND(avg(amount::INTEGER),2) AS loan_average_app,
        ROUND(avg(income::INTEGER),2) AS income_average_app,
        COUNT(concat(agency, rid)) AS count_app,
        SUM(amount::INTEGER) AS value_app,
        --ROUND(((amount::NUMERIC / 0.80) / income::NUMERIC),2) AS income_mult_app,
    CONCAT(state, county) AS fips

        FROM hmdalar2000

        WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND income NOT LIKE '%NA%'
        AND income NOT LIKE '%na% '
        AND amount NOT LIKE '%NA%'
        AND amount NOT LIKE '%na%'

        GROUP BY state, county, year, amount, income),

 county_orig AS (
 SELECT year, state, county,
    ROUND(avg(amount::INTEGER),2) AS loan_average_orig,
        ROUND(avg(income::INTEGER),2) AS income_average_orig,
        COUNT(concat(agency, rid)) AS count_orig,
        SUM(amount::INTEGER) AS value_orig,
        --ROUND(((amount::NUMERIC / 0.80) / income::NUMERIC),2) AS income_mult_orig,
    CONCAT(state, county) AS fips

        FROM hmdalar2000

        WHERE loan_type = '1'
        AND action = '1'
        AND loan_purpose in ('1', '3')
        AND income NOT LIKE '%NA%'
        AND income NOT LIKE '%na% '
        AND amount NOT LIKE '%NA%'
        AND amount NOT LIKE '%na%'
        
        GROUP BY state, county, year, amount, income)

SELECT 
county_apps.state,
county_apps.county,
county_apps.fips,
county_apps.loan_average_app,
county_apps.income_average_app,
county_apps.count_app,
county_apps.value_app,
--county_apps.income_mult_app,
county_orig.loan_average_orig,
county_orig.income_average_orig,
county_orig.count_orig,
county_orig.value_orig,
--county_orig.income_mult_orig,
(county_orig.count_orig::NUMERIC / (county_apps.count_app::NUMERIC + county_orig.count_orig::NUMERIC)) AS orig_rate

FROM county_apps
JOIN county_orig
ON county_orig.fips = county_apps.fips """

    SQL = SQL.format(table=table, type='1')
    #cur.execute(SQL)

    #data = cur.fetchall() #retrieve data from query
    df = pd.read_sql_query(SQL, conn)
    if first:
        out_file = df
        first = False
    else:
        out_file = pd.concat([out_file, df])
    print df.head()


out_file.to_csv('county_aggregates.csv')



IndentationError: expected an indented block (<ipython-input-60-10099e70c0d5>, line 6)

In [47]:
county_agg_df = out_file.copy()

In [49]:
county_agg_df.tail()

,year,state,county,loan_average_app,income_average_app,count_app,value_app
3263,2014,35,045,141.76,86.14,2013,285365
3264,2014,18,141,124.91,86.98,4462,557346
3265,2014,40,001,92.31,61.94,194,17909
3266,2014,29,021,104.07,77.21,1577,164124
3267,2014,48,491,194.24,115.24,18701,3632536


In [52]:
#selects originated loan information for single family homes
first = True
for table in hmda_tables[4:]:
    SQL = """
WITH rates AS (
    SELECT year, state, county,
    CONCAT(state, county) AS fips,
    COUNT(rate_spread) AS has_rs,
    AVG(rate_spread::NUMERIC) AS avg_rate_spread

    FROM {table}
    
    WHERE loan_type = '1'
        AND loan_purpose IN ('1', '3')
        AND property_type = '{type}'
        AND action = '1'
        AND lien = '1'
        AND rate_spread != 'NA   '
        AND rate_spread != '     '
        
    GROUP BY year, state, county),

rates2 as (
    SELECT year, state, county,
    CONCAT(state, county) AS fips,
    COUNT(rate_spread) AS no_rs
    
    FROM {table}
    
    WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND property_type = '{type}'
        AND action = '1'
        AND lien = '1'
        AND rate_spread = 'NA   '
        
GROUP BY year, state, county),

county_agg_orig AS (
    SELECT year, state, county,
    CONCAT(state, county) AS fips,
        ROUND(avg(amount::INTEGER),2) AS loan_average,
        ROUND(avg(income::INTEGER),2) AS income_average,
        COUNT(concat(agency, rid)) AS count_orig,
        SUM(amount::INTEGER) AS value_orig,
        SUM(income::INTEGER) AS total_income_orig

    FROM {table}
    
    WHERE loan_type = '1'
        AND loan_purpose IN ('1', '3')
        AND property_type = '{type}'
        AND action = '1'
        AND lien = '1'
        AND rate_spread != '     '
        AND income NOT LIKE '%NA%'
        AND income NOT LIKE '%na% '
        AND income != '    '
        AND amount != '     '
        AND amount NOT LIKE '%NA%'
        AND amount NOT LIKE '%na%'
        
        GROUP BY state, county, year ),

county_agg_apps AS (
SELECT year, state, county,
        CONCAT(state, county) AS fips,
        ROUND(AVG(amount::INTEGER),2) AS loan_average,
        ROUND(AVG(income::INTEGER),2) AS income_average,
        COUNT(CONCAT(agency, rid)) AS count_app,
        SUM(amount::INTEGER) AS value_app,
        SUM(income::INTEGER) AS total_income_app
        
        FROM {table}
        
        WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND property_type = '{type}'
        AND action != '1'
        AND lien = '1'
        AND rate_spread != '     '
        AND income not like '%NA%'
        AND income not like '%na% '
        AND income != '    '
        AND amount != '     '
        AND amount not like '%NA%'
        AND amount not like '%na%'
        
        GROUP BY state, county, year )
        
SELECT rates.year, rates.state, rates.county, rates.fips, rates.has_rs, rates2.no_rs, 
ROUND(rates.avg_rate_spread, 2) AS avg_rs, 
ROUND(((rates.has_rs::NUMERIC / (rates2.no_rs::NUMERIC + rates.has_rs::NUMERIC))*100),2) AS rs_ratio,
county_agg_orig.loan_average, 
county_agg_orig.income_average, 
county_agg_orig.count_orig, 
county_agg_orig.value_orig,
ROUND((value_orig::NUMERIC / total_income_orig::NUMERIC),2) AS total_mult_orig,
ROUND(((value_app::NUMERIC / 0.80) / total_income_app::NUMERIC),2) AS total_mult_app,
ROUND(((county_agg_apps.loan_average::NUMERIC / 0.80) / county_agg_apps.income_average::NUMERIC),2) AS income_multiple_app,
ROUND(((county_agg_orig.loan_average::NUMERIC /0.80) / county_agg_orig.income_average::NUMERIC),2) AS income_multiple_orig,

county_agg_apps.loan_average AS app_loan_avg,
county_agg_apps.income_average AS app_income_avg,
county_agg_apps.count_app AS count_apps,
county_agg_apps.value_app,
ROUND(((county_agg_orig.count_orig::NUMERIC / (county_agg_orig.count_orig::NUMERIC + county_agg_apps.count_app::NUMERIC))*100),2) AS orig_rate


FROM rates
JOIN rates2
ON concat(rates.state, rates.county) = concat(rates2.state, rates2.county)
JOIN county_agg_orig
ON concat(rates.state, rates.county) = concat(county_agg_orig.state, county_agg_orig.county)
JOIN county_agg_apps
ON concat(rates.state, rates.county) = concat(county_agg_apps.state, county_agg_apps.county);
"""

    SQL = SQL.format(table=table, type='1')
    df = pd.read_sql_query(SQL, conn)
    if first:
        out_file = df
        first = False
    else:
        out_file = pd.concat([out_file,df])
    print df.head()
out_file.to_csv('county_aggregates_post_2004.csv')
#export as single csv for all years
#create table to hold data
#insert data

   year state county  has_rs  no_rs  avg_rs  rs_ratio  loan_average_orig  \
0  2004    01    001     192   1152    4.53     14.29             117.96   
1  2004    01    003     745   7069    4.27      9.53             182.95   
2  2004    01    005     119    222    4.66     34.90              97.44   
3  2004    01    007     101    234    4.49     30.15              89.37   
4  2004    01    009     311   1030    4.42     23.19             110.99   

   income_average_orig  count_orig  value_orig  income_multiple  app_loan_avg  \
0                65.80        1292      152407             2.24        102.70   
1               115.96        7400     1353852             1.97        155.36   
2                67.50         314       30596             1.80        102.91   
3                58.58         319       28510             1.91         95.61   
4                60.90        1298      144062             2.28        101.56   

   app_income_avg  count_app  value_app  orig_rate  
0  

In [57]:

out_file['fips'] = out_file.state + out_file.county
out_file.head()

,year,state,county,has_rs,no_rs,avg_rs,rs_ratio,loan_average_orig,income_average_orig,count_orig,value_orig,income_multiple,app_loan_avg,app_income_avg,count_app,value_app,orig_rate,fips
0,2004,01,001,192,1152,4.53,14.29,117.96,65.80,1292,152407,2.24,102.70,62.25,1280,131458,50.23,01001
1,2004,01,003,745,7069,4.27,9.53,182.95,115.96,7400,1353852,1.97,155.36,92.21,5896,915974,55.66,01003
2,2004,01,005,119,222,4.66,34.90,97.44,67.50,314,30596,1.80,102.91,65.20,505,51972,38.34,01005
3,2004,01,007,101,234,4.49,30.15,89.37,58.58,319,28510,1.91,95.61,57.15,454,43409,41.27,01007
4,2004,01,009,311,1030,4.42,23.19,110.99,60.90,1298,144062,2.28,101.56,63.30,1450,147255,47.23,01009


In [58]:
out_file.groupby('fips').rs_ratio.plot()

/Users/roellk/homebrew/lib/python2.7/site-packages/matplotlib/axes/_base.py:2787: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0.0, right=0.0
  'left=%s, right=%s') % (left, right))


fips
01001    Axes(0.125,0.1;0.775x0.8)
01003    Axes(0.125,0.1;0.775x0.8)
01005    Axes(0.125,0.1;0.775x0.8)
01007    Axes(0.125,0.1;0.775x0.8)
01009    Axes(0.125,0.1;0.775x0.8)
01011    Axes(0.125,0.1;0.775x0.8)
01013    Axes(0.125,0.1;0.775x0.8)
01015    Axes(0.125,0.1;0.775x0.8)
01017    Axes(0.125,0.1;0.775x0.8)
01019    Axes(0.125,0.1;0.775x0.8)
01021    Axes(0.125,0.1;0.775x0.8)
01023    Axes(0.125,0.1;0.775x0.8)
01025    Axes(0.125,0.1;0.775x0.8)
01027    Axes(0.125,0.1;0.775x0.8)
01029    Axes(0.125,0.1;0.775x0.8)
01031    Axes(0.125,0.1;0.775x0.8)
01033    Axes(0.125,0.1;0.775x0.8)
01035    Axes(0.125,0.1;0.775x0.8)
01037    Axes(0.125,0.1;0.775x0.8)
01039    Axes(0.125,0.1;0.775x0.8)
01041    Axes(0.125,0.1;0.775x0.8)
01043    Axes(0.125,0.1;0.775x0.8)
01045    Axes(0.125,0.1;0.775x0.8)
01047    Axes(0.125,0.1;0.775x0.8)
01049    Axes(0.125,0.1;0.775x0.8)
01051    Axes(0.125,0.1;0.775x0.8)
01053    Axes(0.125,0.1;0.775x0.8)
01055    Axes(0.125,0.1;0.775x0.8)
01057    Axes(0